### Install necessary libraries

In [2]:
!pip install transformers datasets accelerate


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Prepare the dataset

In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/for_training_no_nulls.csv')

# Clean and prepare
df.dropna(subset=['title', 'description', 'genre', 'rating'], inplace=True)

df['rating'] = df['rating'].astype(str)
df['genre'] = df['genre'].str.lower().str.strip()

# Create task inputs
df_rating = df.copy()
df_rating['input_text'] = "predict rating: Title: " + df['title'] + " Description: " + df['description']
df_rating['target_text'] = df['rating']

df_genre = df.copy()
df_genre['input_text'] = "predict genre: Title: " + df['title'] + " Description: " + df['description']
df_genre['target_text'] = df['genre']

# Combine both tasks
final_df = pd.concat([df_rating[['input_text', 'target_text']], df_genre[['input_text', 'target_text']]])
final_df = final_df.sample(frac=1).reset_index(drop=True)



### Load and fine tune T5

In [2]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 291.0 kB/s eta 0:00:00a 0:00:01


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize dataset
def preprocess(example):
    input_enc = tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512)
    target_enc = tokenizer(example['target_text'], padding="max_length", truncation=True, max_length=64)
    return {
        'input_ids': input_enc['input_ids'],
        'attention_mask': input_enc['attention_mask'],
        'labels': target_enc['input_ids']
    }

dataset = Dataset.from_pandas(final_df)
tokenized_dataset = dataset.map(preprocess)

# Define training args
training_args = TrainingArguments(
    output_dir="./t5-multitask",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    # evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Train
trainer.train()

Map:   0%|          | 0/102280 [00:00<?, ? examples/s]

/tmp/ipython-input-6-3034974924.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abrarfahim75 (abrarfahim75-hamburg-university-of-applied-sciences) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,12.315700
20,5.759100
30,2.750900
40,1.876700
50,1.578800
60,1.527700
70,1.042300
80,1.194000
90,1.138500
100,1.293400


TrainOutput(global_step=76710, training_loss=0.28855541003075186, metrics={'train_runtime': 21223.6078, 'train_samples_per_second': 14.457, 'train_steps_per_second': 3.614, 'total_flos': 4.152827836366848e+16, 'train_loss': 0.28855541003075186, 'epoch': 3.0})

In [7]:
print("Training completed.")


Training completed.


In [1]:
model.save_pretrained("/content/drive/MyDrive/t5-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/t5-finetuned")
print("Model saved!")


NameError: name 'model' is not defined

### Reusing the training data

In [1]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google'

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model_path = "/content/drive/MyDrive/t5-finetuned"
model_path = "../t5-finetuned"

tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [5]:
import torch

def predict(input_text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
# Example 1: Fantasy novel
print(predict("predict genre: Title: Harry Potter and the Sorcerer's Stone Description: A young boy discovers he is a wizard on his 11th birthday and attends a magical school called Hogwarts, where he makes friends and uncovers the truth about his parents' mysterious deaths."))

print(predict("predict rating: Title: Harry Potter and the Sorcerer's Stone Description: A young boy discovers he is a wizard on his 11th birthday and attends a magical school called Hogwarts, where he makes friends and uncovers the truth about his parents' mysterious deaths."))


# Example 2: Science fiction thriller
print(predict("predict genre: Title: The Martian Description: After a fierce storm forces his crew to evacuate, an astronaut is left stranded on Mars. With limited supplies, he must rely on his ingenuity to survive while awaiting rescue."))

print(predict("predict rating: Title: The Martian Description: After a fierce storm forces his crew to evacuate, an astronaut is left stranded on Mars. With limited supplies, he must rely on his ingenuity to survive while awaiting rescue."))


['fantasy', 'young adult', 'fiction','magic', 'childrens','middle grade', 'adventure', 'young adult fantasy', 'juvenile'
4.03
['science fiction', 'fiction','space opera','space','science fiction fantasy','space','space', 'adventure','space', 'novels', 'audiobook'
3.86


In [1]:
# ✅ Ensure GPU is available
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA Available: False
GPU Name: No GPU


### Contact support TahirBro.com